In [2]:
import descarteslabs as dl
from datetime import datetime
from subprocess import check_output
import subprocess 
import json
import pickle
from datetime import timedelta

## Open File

In [ ]:
process = subprocess.call('mkdir temp', shell=True)

In [16]:
args = ['gsutil cp gs://dl-fires/inciweb-data/conus_fires_20180517-18UTC.txt temp/']
p = subprocess.Popen(args, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print p.communicate()

('', 'Copying gs://dl-fires/inciweb-data/conus_fires_20180517-18UTC.txt...\n/ [0 files][    0.0 B/  1.0 KiB]                                                \r/ [1 files][  1.0 KiB/  1.0 KiB]                                                \r\nOperation completed over 1 objects/1.0 KiB.                                      \n')


In [3]:
def iterate_over_file(string):
    counter = 0
    for line in iter(string.splitlines()):
        if counter == 0:
            counter +=1
        else: 
            initiate_fire_instance(line)
            counter +=1

In [4]:
# This could be refactored to include a class 
def initiate_fire_instance(fire_string): 
    fire_arr = fire_string.split()
    name_list = fire_arr[:-4]
    name = " ".join(name_list)
    fire_id = fire_arr[-4]
    lat = float(fire_arr[-3])
    lon = float(fire_arr[-2])
    coords=[lat,lon]
    acres = fire_arr[-1]
    check_fire_status(fire_id, name, coords, acres)

In [44]:
# CURRENT RUN SCRIPT, though depends on the above being run first 
file_contents = read_latest_conus_file()
iterate_over_file(file_string_contents)

('', 'Copying gs://dl-fires/inciweb-data/conus_fires_20180518-18UTC.txt...\n/ [0 files][    0.0 B/  1.1 KiB]                                                \r/ [1 files][  1.1 KiB/  1.1 KiB]                                                \r\nOperation completed over 1 objects/1.1 KiB.                                      \n')


### Check if fire instance exists 

In [6]:
def check_fire_status(fire_id, name, coordinates, acres):
    fires = get_fire_instances() 
    if fire_id in fires:
        last_checked =  fires[fire_id]['last_checked']
        existing_fire(name, fire_id, coordinates, last_checked)
    else:
        print "New fire"
        new_fire_instance(fire_id, name, coordinates, acres)
        

## Read in latest Inciweb report

In [43]:
def read_latest_conus_file():
    # List all files 
    args = ['gsutil ls gs://dl-fires/inciweb-data/']
    p = subprocess.Popen(args, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    inciweb_data_files = p.communicate()[0]
 
    # Isolate latest report  
    latest_entry = inciweb_data_files.split('\ngs://dl-fires/inciweb-data/')[-1]
    conus_file = latest_entry.replace('\n', '')
    
    # Download and open the latest report     
    args = ['gsutil cp gs://dl-fires/inciweb-data/'+ conus_file +' temp/']
    p = subprocess.Popen(args, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print p.communicate()
    txt = open('temp/'+ conus_file)
    file_string_contents = txt.read()
    return file_string_contents

## Read in fire instances

In [7]:
def get_fire_instances(): 
    with open('temp/fires.pickle', 'rb') as handle:
        b = pickle.load(handle)
    return b

## Logic for existing fire instance or new fire instance

In [8]:
def new_fire_instance(fire_id, name, coordinates, acres): 
    five_days_ago = str(datetime.today() + timedelta(days=-5))
    find_imagery(name, five_days_ago ,coordinates, fire_id) 
    
    # Make bucket for imagery 
    'gs://dl-fires/imagery'+ fire_id
    with open('temp/fires.pickle', 'rb') as handle:
        b = pickle.load(handle)
        b[fire_id] = {
            'name': 'Mallard Fire',
            'date_entered': datetime.today().isoformat()
        }
        
    with open('temp/fires.pickle', 'wb') as handle:
        pickle.dump(b, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [9]:
def existing_fire(name, fire_id, coordinates, last_checked):
#     bucket = 'gs://dl-fires/imagery/'+ fire_id
    find_imagery(name, last_checked ,coordinates, fire_id ) 
    with open('temp/fires.pickle', 'rb') as handle:
        b = pickle.load(handle)
        b[fire_id]['last_checked'] = datetime.today().isoformat()
        
    with open('temp/fires.pickle', 'wb') as handle:
        pickle.dump(b, handle, protocol=pickle.HIGHEST_PROTOCOL)


## Find imagery

In [10]:
def make_tile(coords): 
    # to get the tile centered around your lat/lon, give it a tilesize of 1 and a pad of the half-width of your desired image (in meters)
    print coords[0]
    print coords[1]
    bounding_box = dl.raster.dltile_from_latlon(
        lat=coords[0],
        lon=coords[1], 
        resolution=30, 
        tilesize=2, 
        pad=600)

    return bounding_box
    

In [11]:
def search_products(date, geometry):
    print(date)
    from pprint import pprint 
    pprint(geometry['geometry']['coordinates'])
    images = dl.metadata.search(
        ['landsat:LC08:01:RT:TOAR', 'sentinel-2:L1C'],
        geom= geometry, 
        start_datetime=date, 
        end_datetime= datetime.today().isoformat()
        )
    pprint("The number of total images found is {}.".format(len(images['features'])))
    return [image['id'] for image in images['features']]

In [12]:
def download_imagery(ids, geometry, fire_id):
    for image_id in ids: 
        print image_id
        dl.raster.raster(
                        image_id,
                        bands=['swir2', 'swir1', 'nir', 'alpha'],
                        scales=[[0,4000], [0, 4000], [0, 4000], None],
                        data_type='Byte',
                        cutline=geometry,
                        save= True,
                        outfile_basename="temp/"+image_id)

        # Sync images to associated bucket
        args = ['gsutil cp  temp/'+image_id+ '.tif gs://dl-fires/imagery/'+fire_id+'/']
        p = subprocess.Popen(args, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print p.communicate()
        

In [14]:
def find_imagery(fire_name, date, coords, fire_id, footprint=None):
    # Where name is a str, coords is an array of [lat,long], and footprint is an optional geometry
    # if footprint:
        # image_ids = search_products(date,footprint)
        # download_imagery(image_ids, footprint)
    # else:
    dl_tile = make_tile(coords)
    image_ids = search_products(date, dl_tile)
    download_imagery(image_ids, dl_tile, fire_id)

Driver Code

In [ ]:
fires = {
    'id:5790':{
        'name': 'Mallard Fire',
        'date_entered': '2018-05-10',
        'last_checked': '2018-05-15'
    },
    'id:5795':{
        'name': 'Pinery Fire',
        'date_entered': '2018-05-10',
        'last_checked': '2018-05-15'
    }
}

In [ ]:
with open('temp/fires.pickle', 'wb') as handle:
    pickle.dump(fires, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('temp/fires.pickle', 'rb') as handle:
    b = pickle.load(handle)

print b

In [ ]:
args = ['gsutil ls gs://dl-fires/imagery']
p = subprocess.Popen(args, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
buckets= p.communicate()[0]
buckets = buckets.split('\ngs://dl-fires/imagery/')
buckets = [bucket.replace('/', '') for bucket in buckets]
buckets = [bucket.replace('\n', '') for bucket in buckets]
buckets = buckets[1:]
buckets